## **This notebook aims to compare some models and store them in a reasonable fashion**

In [9]:
import pandas as pd
import numpy as np
import os
import time
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [10]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Use GPU 0 in first notebook

In [11]:
project_dir = "/home/jupyter-tfg2425paula/prediction_project_v3"
os.chdir(project_dir)

clean_data_dir = os.path.join(project_dir, "00_data/clean")
horizontal_data_dir = os.path.join(project_dir, "00_data/horizontal_structure")
results_dir = os.path.join(project_dir, "02_results")
pca_data_dir = os.path.join(project_dir, "00_data/pca")

### **GRU Model**

In [12]:
class GRU3DClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout):
        super(GRU3DClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):

        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.gru(x, h0)
        out = self.fc(out[:, -1, :]) 
        # return self.sigmoid(out)
        return out

### **LSTM Model**

In [13]:
class StockPriceLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=1, dropout=0.0):
        super(StockPriceLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, 
                            batch_first=True, dropout=dropout)
    
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        batch_size = x.size(0)  # Get the batch size dynamically

        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(x.device)  # (num_layers, batch_size, hidden_dim)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(x.device)  # (num_layers, batch_size, hidden_dim)
        
        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :]) 
        # out = self.sigmoid(out)
        return out
    

### **Set folders**

Processing

In [14]:
processing_types = ["clean", "pca"]
processing_types= ["clean"]

Folders

In [15]:
# stocks = ['AAPL', 'MSFT', 'AMZN', 'NVDA', 'SPX']
stocks = ['AAPL']
# types_securities = ["single_name", "options", "technical"]
types_securities = ["options"]

Different files

In [16]:
# years = ["15y", "10y", "5y", "2y"]
years = ["10y"]
# window_sizes = [5, 10, 50, 100]
window_sizes = [5]
# train_sizes = [80, 90, 95]
train_sizes = [95]

Same file

In [17]:
thresholds = [0.3, 0.35, 0.4, 0.45, 0.5]
thresholds = [0.5]
learning_rates = [0.005, 0.008, 0.009, 0.01]
learning_rates = [0.01]
num_epochs_list = [100, 200]
num_epochs_list = [100]
batch_sizes = [16, 32]
batch_sizes = [16]
prediction_thresholds = [0.35, 0.4, 0.45, 0.5]
prediction_thresholds = [0.5]

#### **Model and Hyperparameters**

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

hidden_size = 64  
output_size = 2  
num_layers = 2
dropout = 0.2

criterion = nn.CrossEntropyLoss()

In [23]:
model_types = ["lstm", "gru"]   

#### **Last data modifications**

In [20]:
def reshape_remove_characters(df):

    X = np.array([np.stack(row) for row in df.drop(columns=['Target']).values])
    y = df['Target'].values

    smote = SMOTE(random_state=42)
    n_samples, timesteps, n_features = X.shape
    X_flat = X.reshape((n_samples, timesteps * n_features))
    X_flat = np.where(X_flat == 'ç', 0, X_flat)

    X_resampled = X_flat.reshape((-1, timesteps, n_features))
    
    return X_resampled, y

### **Evaluation function**

In [21]:
def evaluate_rolling_unchanged_model_threshold(
    model, 
    X, 
    y, 
    criterion, 
    optimizer, 
    device, 
    train_size, 
    batch_size, 
    num_epochs, 
    lower_threshold
):
    """
    Evaluate a PyTorch model using a rolling prediction approach for time series,
    training the model only once on the initial training set. For each time step
    after train_size, the model makes a prediction without further parameter updates.
    Only predicts +1 or -1 if the probability of class 1 is above/below given thresholds;
    otherwise, predicts 0. Accuracy is computed only on nonzero predictions.

    Args:
        model:          PyTorch model to evaluate.
        X:              Feature data (numpy array).
        y:              Target data (numpy array).
        criterion:      Loss function (e.g., CrossEntropyLoss).
        optimizer:      Optimizer (e.g., Adam).
        device:         Device for computation (CPU or GPU).
        train_size:     Initial size of the training data (int or float).
                        If < 1, treated as fraction of total length.
        batch_size:     Batch size for training.
        num_epochs:     Number of epochs for initial training only.
        lower_threshold: Probability threshold below which model predicts -1.
        upper_threshold: Probability threshold above which model predicts +1.

    Returns:
        dict: Dictionary with the following keys:
            - "rolling_predictions": All predictions (-1, 0, +1) across the test period.
            - "rolling_targets": Corresponding true targets in [-1, +1].
            - "filtered_predictions": Nonzero predictions only.
            - "filtered_targets": Targets corresponding to nonzero predictions.
            - "accuracy_nonzero": Accuracy computed only on nonzero predictions.
    """

    # Convert X, y to tensors
    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.long)

    # Determine initial training set size
    if train_size < 1.0:
        lower_bound = int(train_size * len(X))
    else:
        lower_bound = train_size

    # -------------------------
    # 1) SINGLE TRAINING PHASE
    # -------------------------
    model.to(device)
    model.train()
    
    X_train = X[:lower_bound].to(device)
    y_train = y[:lower_bound].to(device)

    train_dataset = TensorDataset(X_train, y_train)
    trainloader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=False,         # Keep False if order matters; True for better generalization
        # num_workers=4,         # Adjust based on your CPU cores
        # pin_memory=True,       # Speeds up transfer if using GPUs
        drop_last=False        # Ensure the last batch is included
    )

    epoch_train_losses = []
    for epoch in range(num_epochs):
        # torch.cuda.empty_cache()
        epoch_loss = 0.0
        for X_batch, y_batch in trainloader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()
            pred_y = model(X_batch)   # [batch_size, num_classes]
            loss = criterion(pred_y, y_batch)
            loss.backward()

            # Gradient clipping (optional)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            epoch_loss += loss.item()
               
        if (epoch + 1) % 5 == 0 or epoch == num_epochs - 1:
            print(f"[Train] Epoch {epoch+1}/{num_epochs}, Loss={epoch_loss/len(trainloader):.4f}")

        epoch_train_losses.append(epoch_loss/len(trainloader))
        
    loss_decrease_percentage = ((epoch_train_losses[-1] - epoch_train_losses[0]) / epoch_train_losses[0]) * 100
    # ---------------------------------
    # 2) ROLLING PREDICTIONS, NO UPDATE
    # ---------------------------------
    model.eval()

    rolling_predictions = []
    rolling_targets     = []

    for i in range(lower_bound, len(X)):
        # Single-step "test" sample
        X_test = X[i:i+1].to(device)  # shape: (1, num_features)
        y_test = y[i:i+1].to(device)  # shape: (1, )

        with torch.no_grad():
            # Forward pass
            pred_y = model(X_test)  # [1, num_classes]
            probabilities = torch.softmax(pred_y, dim=1).cpu().numpy()  # shape: (1, 2)
            prob_class_1  = probabilities[:, 1]  # shape: (1,)

            # Threshold-based logic
            # Initialize all predictions to 0
            pred_classes = np.zeros_like(prob_class_1)
            # Predict -1 if prob < lower_threshold
            pred_classes[prob_class_1 < lower_threshold] = -1
            # Predict +1 if prob > upper_threshold
            pred_classes[prob_class_1 > 1-lower_threshold] = 1

        rolling_predictions.append(pred_classes[0])  # scalar
        rolling_targets.append(y_test.item())

    rolling_predictions = np.array(rolling_predictions)
    rolling_targets = np.array(rolling_targets).astype(int)

    # Convert any 0-labeled targets to -1 if your original data is in [-1, +1]
    # (Sometimes y might be {0,1} or {-1, +1}; adapt as needed.)
    rolling_targets[rolling_targets == 0] = -1

    # Filter out zero predictions
    nonzero_mask = rolling_predictions != 0
    filtered_preds = rolling_predictions[nonzero_mask]
    filtered_targets = rolling_targets[nonzero_mask]

    if len(filtered_preds) == 0:
        accuracy_nonzero = None
        print("No nonzero predictions, cannot compute thresholded accuracy.")
    else:
        accuracy_nonzero = accuracy_score(filtered_targets, filtered_preds)
        print(f"Accuracy on Nonzero Predictions: {accuracy_nonzero:.4f}")

    return {
        "rolling_predictions": rolling_predictions,
        "rolling_targets": rolling_targets,
        "filtered_predictions": filtered_preds,
        "filtered_targets": filtered_targets,
        "accuracy_nonzero": accuracy_nonzero,
        "loss_decrease_percentage": loss_decrease_percentage
    }

### **1st Type of comparison:**

In [14]:
# folder
results_list = []
stock = "AAPL"
period = "10y"
possible_train_size = 95
for processing in processing_types:
    initial_data_dir = os.path.join(project_dir, f"00_data/{processing}")
    for model_type in model_types:
        for security_type in types_securities:
            output_folder = os.path.join(results_dir, f"{model_type}/{stock}/{security_type}") 
            os.makedirs(output_folder, exist_ok=True)
            # files
            # load original data as well (for info purposes)
            
            filename = f"{security_type}/{stock}/{period}_data.csv"
            original_input_filepath = os.path.join(initial_data_dir, filename)
            original_data = pd.read_csv(original_input_filepath)
            start_date = original_data.loc[0, "Date"]
            end_date = original_data.iloc[-1]["Date"]
            results_csv_path = os.path.join(output_folder, f"{period}_{possible_train_size}.csv")

            # columns, same file
            for window_size in window_sizes:
                print(f"{stock}, {security_type}, {period}, {possible_train_size}, {window_size}")

                # load data
                pkl_filename = f"clean/{security_type}/{stock}/{period}_{window_size}_data.pkl"
                input_filepath = os.path.join(horizontal_data_dir, pkl_filename)
                print(input_filepath)
                input_df = pd.read_pickle(input_filepath)

                X_resampled, y_resampled = reshape_remove_characters(input_df)

                input_size = X_resampled.shape[2]
                train_size = int(X_resampled.shape[0]*possible_train_size/100)
                test_size = X_resampled.shape[0] - train_size

                # generate model
                if model_type == "gru":
                    model = GRU3DClassifier(input_size, hidden_size, output_size, num_layers, dropout)
                elif model_type == "lstm":
                    model = StockPriceLSTM(input_size, hidden_size, output_size)

                model = model.to(device)

                for learning_rate in learning_rates:

                    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

                    for num_epochs in num_epochs_list:
                        for prediction_threshold in prediction_thresholds:
                            for batch_size in batch_sizes:

                                print(f"Training {stock} | LR: {learning_rate} | Epochs: {num_epochs} | Batch: {batch_size} | Prediction Threshold: {prediction_threshold}")

                                start_time = time.time()

                                result = evaluate_rolling_unchanged_model_threshold(
                                    model, X_resampled, y_resampled, criterion, 
                                                               optimizer, device, train_size, batch_size, num_epochs, lower_threshold=prediction_threshold)     

                                rolling_predictions = result["rolling_predictions"]
                                rolling_targets = result["rolling_targets"]
                                test_accuracy = result["accuracy_nonzero"]
                                loss_decrease_percentage = result["loss_decrease_percentage"]
                                nonzero_preds = np.count_nonzero(result["rolling_predictions"])


                                end_time = time.time()    
                                execution_time = end_time - start_time

                                # --------------------------------------------
                                # 1) Create a record (dictionary) for this run
                                # --------------------------------------------
                                run_record = {
                                    "start_date": start_date,
                                    "end_date": end_date,
                                    "execution_time": execution_time,
                                    "test_size": test_size,
                                    "nonzero_preds": nonzero_preds,
                                    "accuracy": test_accuracy,
                                    "prediction_threshold": prediction_threshold,

                                    "window_size": window_size,
                                    "learning_rate": learning_rate,
                                    "num_epochs": num_epochs,
                                    "train_loss_change_pctg": loss_decrease_percentage,

                                    "batch_size": batch_size,

                                    "output_size": output_size,
                                    "hidden_size": hidden_size,
                                    "num_layers": num_layers,
                                    "dropout_rate": dropout,
                                    "optimizer": optimizer.__class__.__name__,
                                    "criterion": criterion

                                }

                                # --------------------------------------------
                                # 2) Append the dictionary to the results list
                                # --------------------------------------------
                                results_list.append(run_record)

            # ----------------------------------------------------------------
            # 3) Write to CSV *once* after all window_sizes for this setup
            # ----------------------------------------------------------------
            if len(results_list) > 0:
                df = pd.DataFrame(results_list)

                # If CSV already exists, append without header
                if os.path.exists(results_csv_path):
                    df.to_csv(results_csv_path, mode='a', header=False, index=False)
                else:
                    df.to_csv(results_csv_path, index=False)

                # Clear results_list to avoid duplication
                results_list = []


AAPL, options, 15y, 95, 5
/home/jupyter-tfg2425paula/prediction_project_v3/00_data/horizontal_structure/clean/options/AAPL/15y_5_data.pkl
Training AAPL | LR: 0.01 | Epochs: 100 | Batch: 16 | Prediction Threshold: 0.5
[Train] Epoch 5/100, Loss=0.6923
[Train] Epoch 10/100, Loss=0.6754
[Train] Epoch 15/100, Loss=0.6125
[Train] Epoch 20/100, Loss=0.5152
[Train] Epoch 25/100, Loss=0.4226
[Train] Epoch 30/100, Loss=0.3315
[Train] Epoch 35/100, Loss=0.2615
[Train] Epoch 40/100, Loss=0.2094
[Train] Epoch 45/100, Loss=0.1780
[Train] Epoch 50/100, Loss=0.1432
[Train] Epoch 55/100, Loss=0.1050
[Train] Epoch 60/100, Loss=0.1082
[Train] Epoch 65/100, Loss=0.1127
[Train] Epoch 70/100, Loss=0.1077
[Train] Epoch 75/100, Loss=0.0826
[Train] Epoch 80/100, Loss=0.0970
[Train] Epoch 85/100, Loss=0.0864
[Train] Epoch 90/100, Loss=0.0659
[Train] Epoch 95/100, Loss=0.0648
[Train] Epoch 100/100, Loss=0.1268
Accuracy on Nonzero Predictions: 0.5408


In [38]:
results_list = []
stock = "AAPL"
period = "10y"
possible_train_size = 95
window_size = 5
batch_size = 32
num_epochs = 100

prediction_threshold = 0.5

output_folder = os.path.join(results_dir, f"inidividual_trials") 
os.makedirs(output_folder, exist_ok=True)

results_csv_path = os.path.join(output_folder, f"01_{stock}_proc_model_type.csv")
processing_types = ["clean", "pca"]
types_securities = ["technical", "options"]

results_list = []

learning_rates = [0.009, 0.01, 0.011]

for processing in processing_types:  # e.g. ["clean", "pca"]
    initial_data_dir = os.path.join(project_dir, f"00_data/{processing}") 

    for model_type in model_types:
        for security_type in types_securities:
            
            test_accuracy_list = []
            pct_decrease_list = []
            
            # Load original data (info only)
            filename = f"{security_type}/{stock}/{period}_data.csv"
            original_input_filepath = os.path.join(initial_data_dir, filename)
            original_data = pd.read_csv(original_input_filepath)

            # Iterate over window sizes
            print(f"{processing}, {model_type}, {security_type}")

            # Load data using the 'processing' variable in path
            pkl_filename = f"{processing}/{security_type}/{stock}/{period}_{window_size}_data.pkl"
            input_filepath = os.path.join(horizontal_data_dir, pkl_filename)
            print(input_filepath)
            input_df = pd.read_pickle(input_filepath)

            X_resampled, y_resampled = reshape_remove_characters(input_df)

            input_size = X_resampled.shape[2]
            train_size = int(X_resampled.shape[0] * possible_train_size / 100)
            test_size = X_resampled.shape[0] - train_size

            # Generate model
            if model_type == "gru":
                model = GRU3DClassifier(input_size, hidden_size, output_size, num_layers, dropout)
            elif model_type == "lstm":
                model = StockPriceLSTM(input_size, hidden_size, output_size)

            model = model.to(device)
            
            for learning_rate in learning_rates:
                optimizer = optim.Adam(model.parameters(), lr=learning_rate)
                print(f"Training {stock} | LR: {learning_rate} | Epochs: {num_epochs} "
                      f"| Batch: {batch_size} | Prediction Threshold: {prediction_threshold}")

                result = evaluate_rolling_unchanged_model_threshold(
                    model, 
                    X_resampled, 
                    y_resampled, 
                    criterion, 
                    optimizer, 
                    device, 
                    train_size, 
                    batch_size, 
                    num_epochs, 
                    lower_threshold=prediction_threshold
                )     

                rolling_predictions = result["rolling_predictions"]
                rolling_targets = result["rolling_targets"]
                test_accuracy = result["accuracy_nonzero"]
                loss_decrease_percentage = result["loss_decrease_percentage"]
                nonzero_preds = np.count_nonzero(rolling_predictions)
                
                test_accuracy_list.append(test_accuracy)
                pct_decrease_list.append(loss_decrease_percentage)

            print(test_accuracy_list)
            print(pct_decrease_list)
            # 1) Create a record (dictionary) for this run
            run_record = {"STOCK": stock,
                "DATA_TYPE": security_type,
                "MODEL": model_type.upper(),  # Convert to uppercase for consistency
                "PROCESSING": processing,
                "MEAN_ACCURACY": np.mean(test_accuracy_list),
                "MAX_ACCURACY": np.max(test_accuracy_list),
                "MIN_ACCURACY": np.min(test_accuracy_list),
                "MEAN_TRAIN_PCT_DECREASE": np.mean(test_accuracy_list)}

            # 2) Append to list
            results_list.append(run_record)

clean, lstm, technical
/home/jupyter-tfg2425paula/prediction_project_v3/00_data/horizontal_structure/clean/technical/AAPL/10y_5_data.pkl
Training AAPL | LR: 0.009 | Epochs: 100 | Batch: 32 | Prediction Threshold: 0.5
[Train] Epoch 5/100, Loss=0.6928
[Train] Epoch 10/100, Loss=0.6901
[Train] Epoch 15/100, Loss=0.6520
[Train] Epoch 20/100, Loss=0.5889
[Train] Epoch 25/100, Loss=0.5191
[Train] Epoch 30/100, Loss=0.3657
[Train] Epoch 35/100, Loss=0.3002
[Train] Epoch 40/100, Loss=0.2570
[Train] Epoch 45/100, Loss=0.1730
[Train] Epoch 50/100, Loss=0.1429
[Train] Epoch 55/100, Loss=0.1172
[Train] Epoch 60/100, Loss=0.1050
[Train] Epoch 65/100, Loss=0.0820
[Train] Epoch 70/100, Loss=0.0898
[Train] Epoch 75/100, Loss=0.1006
[Train] Epoch 80/100, Loss=0.0774
[Train] Epoch 85/100, Loss=0.0783
[Train] Epoch 90/100, Loss=0.0950
[Train] Epoch 95/100, Loss=0.0630
[Train] Epoch 100/100, Loss=0.0670
Accuracy on Nonzero Predictions: 0.4921
Training AAPL | LR: 0.01 | Epochs: 100 | Batch: 32 | Prediction

In [36]:
df = pd.DataFrame(results_list)
df

,STOCK,DATA_TYPE,MODEL,PROCESSING,MEAN_ACCURACY,MAX_ACCURACY,MIN_ACCURACY
0,AAPL,technical,LSTM,clean,0.479365,0.492063,0.460317
1,AAPL,options,LSTM,clean,0.503817,0.511450,0.480916
2,AAPL,technical,GRU,clean,0.534921,0.571429,0.507937
3,AAPL,options,GRU,clean,0.545038,0.564885,0.496183
4,AAPL,technical,LSTM,pca,0.538095,0.595238,0.452381
5,AAPL,options,LSTM,pca,0.497710,0.541985,0.458015
6,AAPL,technical,GRU,pca,0.482540,0.555556,0.380952
7,AAPL,options,GRU,pca,0.490076,0.526718,0.450382


In [29]:
# 3) Write to CSV once for this batch of window_sizes (avoid partial duplication)
if len(results_list) > 0:
    df = pd.DataFrame(results_list)

    if os.path.exists(results_csv_path):
        # Append without header
        df.to_csv(results_csv_path, mode='a', header=False, index=False)
    else:
        # Write new file with header
        df.to_csv(results_csv_path, index=False)

    # Clear the list before next iteration
    results_list = []